In [1]:
import pymongo
import pandas as pd
from collections import Counter
import random

# Connexion à MongoDB
uri = "mongodb://mongodb:27017/"
client = pymongo.MongoClient(uri)
db = client["github_issues"]
collection = db["closed_issues"]

# Charger toutes les issues dans un DataFrame
issues = list(collection.find())
df = pd.DataFrame(issues)

# Analyse du nombre d'issues par langage
df_language_count = df['language'].value_counts().reset_index()
df_language_count.columns = ['language', 'count']
print(df_language_count)



            language  count
0         JavaScript  28369
1                  C  22401
2         TypeScript   3806
3               Dart   1785
4               Rust   1749
5                MDX   1577
6             Svelte   1174
7              Swift    573
8                Lua    521
9               Java    470
10  Jupyter Notebook    388
11            Python    235
12                       78


In [2]:
# Trouver la valeur min et max de duration
min_duration = collection.find_one({}, sort=[("duration", 1)])["duration"]
max_duration = collection.find_one({}, sort=[("duration", -1)])["duration"]

# Compter les occurrences de chaque valeur de duration
duration_counts = collection.aggregate([
    {"$group": {"_id": "$duration", "count": {"$sum": 1}}},
    {"$sort": {"_id": 1}}
])

# Afficher les résultats
print(f"Durée minimale : {min_duration}")
print(f"Durée maximale : {max_duration}")
print("Occurrences de chaque valeur de duration :")
for entry in duration_counts:
    print(f"Durée {entry['_id']}: {entry['count']} fois")

Durée minimale : 0
Durée maximale : 1306
Occurrences de chaque valeur de duration :
Durée 0: 12447 fois
Durée 1: 6147 fois
Durée 2: 4944 fois
Durée 3: 4466 fois
Durée 4: 4246 fois
Durée 5: 4008 fois
Durée 6: 4029 fois
Durée 7: 3967 fois
Durée 8: 3883 fois
Durée 9: 456 fois
Durée 10: 438 fois
Durée 11: 399 fois
Durée 12: 376 fois
Durée 13: 390 fois
Durée 14: 329 fois
Durée 15: 298 fois
Durée 16: 265 fois
Durée 17: 276 fois
Durée 18: 289 fois
Durée 19: 260 fois
Durée 20: 251 fois
Durée 21: 263 fois
Durée 22: 202 fois
Durée 23: 176 fois
Durée 24: 196 fois
Durée 25: 174 fois
Durée 26: 160 fois
Durée 27: 182 fois
Durée 28: 150 fois
Durée 29: 141 fois
Durée 30: 117 fois
Durée 31: 128 fois
Durée 32: 133 fois
Durée 33: 122 fois
Durée 34: 137 fois
Durée 35: 141 fois
Durée 36: 100 fois
Durée 37: 122 fois
Durée 38: 83 fois
Durée 39: 97 fois
Durée 40: 105 fois
Durée 41: 107 fois
Durée 42: 86 fois
Durée 43: 95 fois
Durée 44: 80 fois
Durée 45: 95 fois
Durée 46: 97 fois
Durée 47: 95 fois
Durée 48: 11

In [6]:
import pymongo
import pandas as pd
from collections import Counter
import random

# Connexion à MongoDB
uri = "mongodb://mongodb:27017/"
client = pymongo.MongoClient(uri)
db = client["github_issues"]
collection = db["closed_issues"]

# Fonction pour attribuer la classe 'duration_class'
def assign_duration_class(duration):
    if duration == 0:
        return 1
    elif duration in [1, 2]:
        return 2
    elif duration in [3, 4, 5]:
        return 3
    elif duration in [6, 7, 8]:
        return 4
    else:
        return 5

# Mettre à jour chaque document dans MongoDB
for issue in collection.find():
    # Calculer la classe 'duration_class' pour chaque issue
    duration_class = assign_duration_class(issue.get('duration', 0))
    
    # Mettre à jour le document avec le nouveau champ 'duration_class'
    collection.update_one(
        {'_id': issue['_id']},
        {'$set': {'duration_class': duration_class}}
    )

# Vérifier l'ajout du nouveau champ
updated_issues = collection.find({}, {'_id': 1, 'duration': 1, 'duration_class': 1}).limit(5)
for issue in updated_issues:
    print(issue)


{'_id': ObjectId('67b24e4fd9191f3304d1f6d0'), 'duration': 2, 'duration_class': 2}
{'_id': ObjectId('67b24e4fd9191f3304d1f6d4'), 'duration': 51, 'duration_class': 5}
{'_id': ObjectId('67b24e4fd9191f3304d1f6d7'), 'duration': 144, 'duration_class': 5}
{'_id': ObjectId('67b24e4fd9191f3304d1f6db'), 'duration': 47, 'duration_class': 5}
{'_id': ObjectId('67b24e4fd9191f3304d1f6de'), 'duration': 161, 'duration_class': 5}


In [8]:
# Vérifier que tous les documents ont le champ 'duration_class'
missing_duration_class = collection.count_documents({"duration_class": {"$exists": False}})

if missing_duration_class > 0:
    print(f"Il y a {missing_duration_class} documents sans 'duration_class'.")
else:
    print("Tous les documents ont le champ 'duration_class'.")


Tous les documents ont le champ 'duration_class'.


In [10]:
# Compter le nombre total de documents
total_count = collection.count_documents({})

# Compter le nombre de documents pour chaque 'duration_class'
duration_class_counts = collection.aggregate([
    {"$group": {"_id": "$duration_class", "count": {"$sum": 1}}}
])

# Calculer et afficher le pourcentage pour chaque 'duration_class'
print("Pourcentage de chaque 'duration_class' :")
for entry in duration_class_counts:
    duration_class = entry['_id']
    count = entry['count']
    percentage = (count / total_count) * 100
    print(f"Class {duration_class}: {percentage:.2f}%")


Pourcentage de chaque 'duration_class' :
Class 4: 18.82%
Class 3: 20.15%
Class 2: 17.57%
Class 5: 23.74%
Class 1: 19.72%
